In [16]:
import pandas as pd
import numpy as np
import sys

In [17]:
#filein = sys.argv[-1:]
filein = 'run_it019_data.star'
fileout = 'clean_data'
data = pd.read_csv(filein, header = 35, sep='\s+|\t+|\n+', engine = 'python', usecols=[2,28], names=['helixID','2Dclass'])

In [18]:
summa = 0
helix = 0
new_helix = pd.DataFrame()
for idx in data['helixID'].index:
    if data.iloc[idx-1,0] <= data.iloc[idx,0]:
        summa = data.iloc[idx,0] + helix - 1
        new_helix = new_helix.append([summa], ignore_index=True)
    else:    
        helix = data.iloc[idx,0] + summa
        new_helix = new_helix.append([helix], ignore_index=True)


In [21]:
data['helixNew']= new_helix
data = data.drop(['helixID'], axis=1)

In [22]:
data = data.drop_duplicates().reset_index(drop=True)
data.head()

,2Dclass,helixNew
0,97,1
1,71,1
2,71,2
3,97,2
4,57,2


In [24]:
n = 1
node1 = pd.DataFrame()
node2 = pd.DataFrame()
network = pd.DataFrame()

for idx in data['helixNew'].index:
    if data.iloc[idx-1,1] == data.iloc[idx,1]:
        node1 = node1.append([data.iloc[idx,0]], ignore_index=True)
        node2 = node2.append([data.iloc[idx-1,0]],ignore_index=True )

In [25]:
node1.head(), node2.head()

(    0
 0  71
 1  97
 2  57
 3  42
 4  71,     0
 0  97
 1  71
 2  97
 3  97
 4  42)

In [26]:
network = pd.concat([node1, node2], axis = 1)
network.columns = ['node1','node2']
network = network.groupby(network.columns.tolist()).size().reset_index().rename(columns={0:'count'})


In [27]:
network.to_csv(fileout, sep='\t', index=False)